In [265]:
# Importing all the necessary libraries we will be needing to do the Ananlysis

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

import urllib # for working with URLs


print('Libraries imported.')

Libraries imported.


## DATA SCRAPING

In [266]:
def createDataFrame(columns, rows):
    dataRows = []
    for tr in rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        if row:
            dataRows.append(row)

    dataFrame = pd.DataFrame(dataRows, columns = columns)
    return dataFrame

In [267]:
# We get the html contain of US cities by population
source_US_Pop =urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population').read()
soup_US_Pop = BeautifulSoup(source_US_Pop, 'lxml')
# get all the row of the table containing popution data
table_US_Pop = soup_US_Pop.find_all('table')[4]
table_rows_US_Pop = table_US_Pop.find_all('tr')

# We get the html contain of the List of United States counties by per capita income
source_IPC =urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_United_States_counties_by_per_capita_income').read()
soup_IPC = BeautifulSoup(source_IPC, 'lxml')
# get all the row of the table containing the capita income data
div_table_IPC = soup_IPC.find('div', class_='mw-parser-output')
table_IPC = div_table_IPC.find('table', class_='wikitable sortable')
table_rows_IPC = table_IPC.find_all('tr')

# Creation du dataFrame
columns_US_population = ["Rank", "City", "State", "Estime", "Census", "Change", "Land_area_mi", "Land_area_km", "Polulation_mi", "Population_km","Location"]
columns_Per_Capita_income =["Rank","Country", "State_federal_district", "Per_capita_income", "Median_household_income", "Median_family_income", "Population", "Number_household"]

dataFrame_US_Pop = createDataFrame(columns_US_population, table_rows_US_Pop)
dataFrame_IPC = createDataFrame(columns_Per_Capita_income, table_rows_IPC)

In [268]:
dataFrame_US_Pop.head()

,Rank,City,State,Estime,Census,Change,Land_area_mi,Land_area_km,Polulation_mi,Population_km,Location
0,1,New York[d],New York,"8,398,748","8,175,133",+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,4,Houston[3],Texas,"2,325,502","2,100,263",+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
4,5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...


In [269]:
dataFrame_IPC.head()

,Rank,Country,State_federal_district,Per_capita_income,Median_household_income,Median_family_income,Population,Number_household
0,1,New York County,New York,"$62,498","$69,659","$84,627","1,605,272","736,192"
1,2,Arlington,Virginia,"$62,018","$103,208","$139,244","214,861","94,454"
2,3,Falls Church City,Virginia,"$59,088","$120,000","$152,857","12,731","5,020"
3,4,Marin,California,"$56,791","$90,839","$117,357","254,643","102,912"
4,5,Alexandria City,Virginia,"$54,608","$85,706","$107,511","143,684","65,369"


## DATA PROCESSING

In [270]:
# We first calculate the radius of every location
# It is equal to the square root of the area.
new_area = dataFrame_US_Pop['Land_area_mi'].str.split("s", n=1, expand = True)
new_area = new_area[0].str.replace(u'\xa0', u'')
dataFrame_US_Pop['Land_area_mi'] =  new_area.str.replace(',','')
dataFrame_US_Pop['Land_area_mi'] =  dataFrame_US_Pop['Land_area_mi'].astype(float)
dataFrame_US_Pop['Radius'] = np.sqrt(dataFrame_US_Pop['Land_area_mi'])

In [271]:
# Furthermore, the latittude and the longitude of every city were extracted.
# It will be used for the visualization
latitude = []
longitude = []
copy_dataFrame_US_Pop = dataFrame_US_Pop.copy(deep = True)
copy_dataFrame_US_Pop['Location'] = copy_dataFrame_US_Pop['Location'].str.split("/", n = 2, expand = True)[1]
new_location = copy_dataFrame_US_Pop['Location'].str.split(" ", n = 0, expand = False)

for i in range(len(new_location)):
    latitude.append(new_location[i][1][:-2])
    longitude.append(new_location[i][2][:-3])

copy_dataFrame_US_Pop["Latitude"] = latitude
copy_dataFrame_US_Pop["Longitude"] = longitude

copy_dataFrame_US_Pop["Latitude"] = copy_dataFrame_US_Pop["Latitude"].str.replace(u'\ufeff', u'')
copy_dataFrame_US_Pop.drop(columns='Location', inplace = True)

copy_dataFrame_US_Pop["Latitude"] = copy_dataFrame_US_Pop["Latitude"].astype(float)
copy_dataFrame_US_Pop["Longitude"] = - copy_dataFrame_US_Pop["Longitude"].astype(float)
copy_dataFrame_US_Pop["Radius"] = copy_dataFrame_US_Pop["Radius"] * 1000
dataFrame_US_Pop = copy_dataFrame_US_Pop.copy(deep=True)

# we delete the unnecessaire columns from the dataframes
dataFrame_US_Pop.drop(columns = ["Rank", "Estime", "Census", "Change", "Land_area_mi", "Land_area_km", "Polulation_mi"], inplace = True)
dataFrame_IPC.drop(columns = ["Rank", "Median_household_income", "Median_family_income", "Number_household"], inplace = True)

In [272]:
dataFrame_US_Pop.head()

,City,State,Population_km,Radius,Latitude,Longitude
0,New York[d],New York,"10,933/km2",17363.755354,40.6635,-73.9387
1,Los Angeles,California,"3,276/km2",21649.480363,34.0194,-118.4108
2,Chicago,Illinois,"4,600/km2",15076.471736,41.8376,-87.6818
3,Houston[3],Texas,"1,395/km2",25248.762346,29.7866,-95.3909
4,Phoenix,Arizona,"1,200/km2",22750.824161,33.5722,-112.0901


In [273]:
dataFrame_IPC.head()

,Country,State_federal_district,Per_capita_income,Population
0,New York County,New York,"$62,498","1,605,272"
1,Arlington,Virginia,"$62,018","214,861"
2,Falls Church City,Virginia,"$59,088","12,731"
3,Marin,California,"$56,791","254,643"
4,Alexandria City,Virginia,"$54,608","143,684"


## Data visualization

In [274]:
# create map of USA cities that we have using latitude and longitude values
map_tohood = folium.Map(location=[37.0902,-95.7129], zoom_start=3)

# add markers to map
for lat, lng, state, city in zip(dataFrame_US_Pop['Latitude'], dataFrame_US_Pop['Longitude'], dataFrame_US_Pop['State'], dataFrame_US_Pop['City']):
    label = '{}, {}'.format(city, state)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_tohood)  
    
map_tohood

### Define Four Square Credentials and version

In [275]:
CLIENT_ID = '1RG1AO5DNXESLXHST3O1P2I150JP2J2XX2VULYGDBV5VXEZX' # your Foursquare ID
CLIENT_SECRET = 'IXCFOZOF23NHFUBZZI3J0YQFPYRU3WWLWNAZAK0WZTCD2LMK' # your Foursquare Secret
VERSION = '20190604'
LIMIT = 20

In [276]:
# Extracts necessary columns into a data frame from the json files that we get when we search using four square API
def getVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng,radius in zip(names, latitudes, longitudes,radius):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
       # print(results)
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


# Getting all the venues of a giving city using four square API 
def getVenuesCity(name, latitudes, longitudes, radius):

    LIMIT = 150       
        # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitudes, 
            longitudes, 
            radius, 
            LIMIT)
            
        # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
   # print(results)
    venues_list=[]
    venues_list.append([(name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name'])for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 'Latitude', 'Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude','Venue Category']
    return(nearby_venues)


In [277]:
# We get the venues of all the cities we got form wikipedia
dataframe_venues = getVenues(names = dataFrame_US_Pop['City'], latitudes = dataFrame_US_Pop['Latitude'],longitudes = dataFrame_US_Pop['Longitude'], radius = dataFrame_US_Pop['Radius'])
dataframe_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York[d],40.6635,-73.9387,Brooklyn Botanic Garden,40.667622,-73.963191,Botanical Garden
1,New York[d],40.6635,-73.9387,Covenhoven,40.675143,-73.960203,Beer Bar
2,New York[d],40.6635,-73.9387,Brooklyn Museum,40.671521,-73.963677,Art Museum
3,New York[d],40.6635,-73.9387,Kings Theatre,40.646110,-73.957175,Theater
4,New York[d],40.6635,-73.9387,Prospect Park (Nethermead),40.660717,-73.968587,Field


#### Assigning weight to revelant venues category according to Armando wish

In [278]:
#Armando want his center to be closed to neighborhood and near activity area such as park, playground etc
#So we have assigned a high values to the category neighboor playground and park.
weights_dict={'Gym':-1,'Gym / Fitness Center':-1,'Climbing Gym':-1,'Hotel':2, 'Playground':3, 'Neighborhood':6, 'Spa':-1, 'Gymnastics Gym':-1}
venues_category = dataframe_venues['Venue Category'] 
list_venues = venues_category.tolist() 

weights = []
for i in list_venues:
    if i in weights_dict.keys():
        weights.append(weights_dict[i])
    else :
        weights.append(0)
dataframe_venues['Weights'] = weights;
dataframe_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Weights
0,New York[d],40.6635,-73.9387,Brooklyn Botanic Garden,40.667622,-73.963191,Botanical Garden,0
1,New York[d],40.6635,-73.9387,Covenhoven,40.675143,-73.960203,Beer Bar,0
2,New York[d],40.6635,-73.9387,Brooklyn Museum,40.671521,-73.963677,Art Museum,0
3,New York[d],40.6635,-73.9387,Kings Theatre,40.646110,-73.957175,Theater,0
4,New York[d],40.6635,-73.9387,Prospect Park (Nethermead),40.660717,-73.968587,Field,0


In [279]:
# Dropping the rows that we are not giving any weight
dataframe_venues.drop(dataframe_venues[dataframe_venues.Weights < 1.0].index, inplace=True)
dataframe_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Weights
37,Los Angeles,34.0194,-118.4108,Beverly Wilshire Hotel (A Four Seasons Hotel) ...,34.066402,-118.400691,Hotel,2
39,Los Angeles,34.0194,-118.4108,The Peninsula Beverly Hills,34.066100,-118.410547,Hotel,2
57,Chicago,41.8376,-87.6818,Soho House,41.883793,-87.648362,Hotel,2
130,San Antonio,29.4724,-98.5251,Hotel EMMA,29.443095,-98.480806,Hotel,2
241,Fort Worth,32.7815,-97.3467,Fort Worth Stockyards National Historic District,32.788794,-97.346710,Neighborhood,6


#### We then group all the venues by city. The weight mean of each city were calculates
#### The population data collected from wikipedia is then merged to the weights cities 

In [280]:
city_weights = dataframe_venues[['City','Weights']].copy()
venues_weights_bycity_means = city_weights.groupby(['City']).mean()
venues_weights_bycity_means = venues_weights_bycity_means.reset_index(drop=False)
venues_weights_bycity_means.head()

,City,Weights
0,Albuquerque,2
1,Allentown,2
2,Ann Arbor,2
3,Athens[z],2
4,Carlsbad,2


In [281]:
city_weight_merged = pd.merge(dataFrame_US_Pop, venues_weights_bycity_means, on='City')
city_weight_merged = city_weight_merged[['City','Population_km','Weights']].copy()
city_weight_merged.head()

,City,Population_km,Weights
0,Los Angeles,"3,276/km2",2
1,Chicago,"4,600/km2",2
2,San Antonio,"1,250/km2",2
3,Fort Worth,962/km2,6
4,Columbus,"1,520/km2",2


#### Data normalization

In [282]:
# Preprocessing the population density in Km2 column as we have to normalize these values
copy_data_city_merged = city_weight_merged.copy(deep = True)
copy_data_city_merged['Population_km'] = copy_data_city_merged['Population_km'].str.split("/", n = 0, expand = True)
copy_data_city_merged['Population_km'] = copy_data_city_merged['Population_km'].str.replace(',','')
copy_data_city_merged['Population_km'] = copy_data_city_merged['Population_km'].astype(float)
city_weight_merged = copy_data_city_merged.copy(deep = True)
city_weight_merged.head()

,City,Population_km,Weights
0,Los Angeles,3276.0,2
1,Chicago,4600.0,2
2,San Antonio,1250.0,2
3,Fort Worth,962.0,6
4,Columbus,1520.0,2


In [283]:
# Normalizing the data frame
from sklearn import preprocessing
normalized_colums = ['Population_km', 'Weights']
x = city_weight_merged[normalized_colums].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
city_weight_merged[normalized_colums] = pd.DataFrame(x_scaled)
city_weight_merged.head()

,City,Population_km,Weights
0,Los Angeles,0.696609,0.0
1,Chicago,1.000000,0.0
2,San Antonio,0.232356,0.0
3,Fort Worth,0.166361,1.0
4,Columbus,0.294225,0.0


#### After the normalization, we calculate the sum of the weight of every city. The city with the high weight value will be selected. The capita incom of this city has to be >= 30.000$ to be considered as the best city for the gym installation

In [284]:
#calculating the sum of normalized columns to determine the city that has maximum sum and conclude that one locality in that city would be the best fit 
city_weight_merged['sum'] = city_weight_merged['Population_km'] + city_weight_merged['Weights']
row_num = np.argmax(np.array(city_weight_merged['sum']))
best_city_name = city_weight_merged['City'].iloc[row_num]
best_city_name

'Detroit'

In [285]:
# Finding the state in which that city belongs
row = dataFrame_US_Pop.loc[dataFrame_US_Pop['City']== best_city_name].index[0]
state_name = dataFrame_US_Pop['State'].iloc[row]
state_name

'Michigan'

In [286]:
# checking the percapita income of Michigan 
p_row = dataFrame_IPC.loc[dataFrame_IPC['State_federal_district'] == state_name].index[0]
per_capital_income = dataFrame_IPC['Per_capita_income'].iloc[p_row]
print("Per capita income of Michigan is :", per_capital_income)

Per capita income of Michigan is : $36,458


### We will now looking for the location of michigan that can be the best one for Armando
* We will use  four square API to get all the venues of Michigan
* Then we will assign a weight to the category of the venues

In [288]:
# Getting coordinates of Michigan
lat_Michigan = dataFrame_US_Pop['Latitude'].iloc[row]
long_Michigan = dataFrame_US_Pop['Longitude'].iloc[row]
print(lat_Michigan, long_Michigan)

42.383 -83.1022


In [289]:
Michigan_venues = getVenuesCity(name = 'Michigan', latitudes = lat_Michigan ,longitudes = long_Michigan, radius = 2500)
Michigan_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Michigan,38.3539,-121.9728,Boston-Edison Historic District,42.378287,-83.099641,Neighborhood
1,Michigan,38.3539,-121.9728,Detroit Repertory Theatre,42.395105,-83.109820,Theater
2,Michigan,38.3539,-121.9728,Motown Historical Museum / Hitsville U.S.A.,42.364246,-83.088574,History Museum
3,Michigan,38.3539,-121.9728,Nandi's Knowledge Cafe,42.398508,-83.092246,Café
4,Michigan,38.3539,-121.9728,Pure Detroit,42.370326,-83.077474,Clothing Store


In [290]:
#copy_michigan_venues = Michigan_venues.copy(deep = True)
#copy_michigan_venues.shape

In [291]:
#copying the data frame in and giving weights for each category
k = Michigan_venues.copy(deep = True)
new_weightage_dict= {'Neighborhood' : 5, 
'Theater':1,
'History Museum':1,
'Café':2,
'Clothing Store':1,
'American Restaurant':1,
'Wings Joint':1,
'Pharmacy':1,
'French Restaurant':1,
'Discount Store':1,
'Fast Food Restaurant':2,
'Bank':1,
'Sandwich Place':1,
'Hardware Store':1,                 
'Coffee Shop':2,                
'Rental Car Location':1,
'Pizza Place':2,
'Fried Chicken Joint':1,
'Grocery Store':1,
'Shoe Store':1,
'Fish Market':1,
'Intersection':4,
'Cosmetics Shop':1,
'Liquor Store':1,
'Clothing Store':1,
'Seafood Restaurant':1,
'Football Stadium':3,
'Check Cashing Service':1,
'Convenience Store':1}


In [292]:
# Calculating new weights for our data frame as we have given weights for all categories
allVenuesinCity1 = k['Venue Category']

f_weights1 = []
for i in allVenuesinCity1:
    if i in new_weightage_dict.keys():
        f_weights1.append(new_weightage_dict[i])
    else :
        f_weights1.append(0)
k['weights'] = f_weights1;
k.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
0,Michigan,38.3539,-121.9728,Boston-Edison Historic District,42.378287,-83.099641,Neighborhood,5
1,Michigan,38.3539,-121.9728,Detroit Repertory Theatre,42.395105,-83.109820,Theater,1
2,Michigan,38.3539,-121.9728,Motown Historical Museum / Hitsville U.S.A.,42.364246,-83.088574,History Museum,1
3,Michigan,38.3539,-121.9728,Nandi's Knowledge Cafe,42.398508,-83.092246,Café,2
4,Michigan,38.3539,-121.9728,Pure Detroit,42.370326,-83.077474,Clothing Store,1


In [293]:
# Dropping unnecessary columns 
newframe = k[['City','Venue Category','weights']].copy()
newframe = k.groupby(['Venue Category']).mean()
newframe.drop(columns = ["Latitude", "Longitude"], inplace = True)
newframe 

,Venue Latitude,Venue Longitude,weights
Venue Category,,,
American Restaurant,42.387949,-83.085379,1
Bank,42.389939,-83.108356,1
Café,42.398508,-83.092246,2
Check Cashing Service,42.375057,-83.076550,1
Clothing Store,42.373479,-83.077301,1
Coffee Shop,42.367385,-83.085375,2
Convenience Store,42.365428,-83.087018,1
Cosmetics Shop,42.392249,-83.125200,1
Discount Store,42.383820,-83.104139,1


#### Plotting all the venues that we have got from the Four Square API

In [301]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

# create map of the venues that we have using latitude and longitudes
venues_map = folium.Map(location=[lat_Michigan, long_Michigan], zoom_start=15) # generate map centred around Jersey city
venues_map.save("venues.png")

# add Michigan City as a red circle mark
folium.features.CircleMarker(
    [lat_Michigan, long_Michigan],
    radius=10,
    popup='Michigan',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add all the venuew of the Michigan to the map as blue circle markers
for lat, lng, label in zip(copy_michigan_venues['Venue Latitude'], copy_michigan_venues['Venue Longitude'], copy_michigan_venues['Venue']):
    label=folium.Popup(label,parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html = False).add_to(venues_map)
    
venues_map

### We cluster every venue using K means algorithm and calculate the weights of each of them

In [306]:
# Cluster them using K means algorithm 
from scipy import stats
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
#Standardize
clmns = ['weights','Venue Latitude', 'Venue Longitude']
df_tr_std = stats.zscore(newframe[clmns])
#Cluster the data
kmeans = KMeans(n_clusters=3, random_state=0).fit(df_tr_std)
labels = kmeans.labels_
newframe['clusters'] = labels
#Add the column into our list
clmns.extend(['clusters'])
#Lets analyze the clusters
kframe = newframe[clmns].groupby(['Venue Category']).mean()
kframe = kframe.reset_index(drop = False)
kframe.head()

,Venue Category,weights,Venue Latitude,Venue Longitude,clusters
0,American Restaurant,1,42.387949,-83.085379,1
1,Bank,1,42.389939,-83.108356,1
2,Café,2,42.398508,-83.092246,1
3,Check Cashing Service,1,42.375057,-83.076550,0
4,Clothing Store,1,42.373479,-83.077301,0


In [303]:
#new group by clusters and add weights of each cluster 
finalWeight = kframe.groupby(['clusters']).mean()
finalWeight

,weights,Venue Latitude,Venue Longitude
clusters,,,
0,1.1,42.371030,-83.081818
1,1.2,42.391904,-83.102928
2,4.0,42.381359,-83.116286


### The location with the maximum weight is ploted with a circle of 50M. The Gym / fitness center of Armando can be installed within this circle

In [304]:
# Final coordinates of the place where we will be setting up an arcade is the one that has maximum weight for, in the above data frame
lat1 = 42.381359  
long1 = -83.116286
# create map of the venues that we have using latitude and longitudes
final_map = folium.Map(location=[lat1, long1], zoom_start=15) # generate map centred around Jersey city

# add prefered location in the City as a green circle mark
folium.features.CircleMarker(
    [lat1, long1],
    radius=50,
    popup='Gym / fitness center can be installed within this circle',
    fill=True,
    color='pink',
    fill_color='pink',
    fill_opacity=0.6
    ).add_to(final_map)
final_map.save("final_map.png")
final_map

### So, we finally got a better place in the Michigan

### This place is between This place is between Webb Street, Linwood Avenue and Lawton Street.